# Google Colab에서 Git Clone을 통한 프로젝트 사용 가이드

이 노트북은 Google Colab에서 GitHub 리포지토리를 클론하여 머신러닝 모델을 학습하는 방법을 설명합니다. 로컬 개발 환경 없이도 Colab의 GPU 리소스를 활용할 수 있습니다.

## 1. Colab 환경 설정

Google Colab에서 프로젝트를 실행하기 위한 기본 설정:

1. **런타임 타입 설정**
   - 런타임 > 런타임 유형 변경 > GPU 선택
   - 고사양 RAM 선택 (필요시)

2. **필요한 라이브러리 설치**
   - PyTorch, torchvision 등 ML 라이브러리
   - 이미지 처리 라이브러리

3. **Google Drive 연결** (선택사항)
   - 데이터셋이 Drive에 저장된 경우

## 2. GitHub 리포지토리 브랜치 클론

**개발 중인 코드를 테스트하기 위해 특정 브랜치에서 클론**:

1. **브랜치 클론 옵션**:
   - 특정 브랜치 직접 클론: `git clone -b branch_name`
   - 모든 브랜치 클론 후 체크아웃
   - 현재 개발 브랜치: `feature/wooseok-baseline-v0`

2. **브랜치 확인 및 전환**:
   - 현재 브랜치 확인
   - 다른 브랜치로 전환
   - 최신 변경사항 업데이트

3. **의존성 설치**:
   - 필요한 Python 패키지 설치
   - 프로젝트 구조 확인

In [ ]:
# 1. 기본 라이브러리 설치
!pip install torch torchvision pillow

# 2. 개발 중인 브랜치에서 직접 클론 (권장)
BRANCH_NAME = "feature/wooseok-baseline-v0"  # 현재 개발 브랜치
print(f"🌿 클론할 브랜치: {BRANCH_NAME}")

!git clone -b {BRANCH_NAME} https://github.com/slay-jini/sprint-ai03-team04.git

# 3. 프로젝트 디렉토리로 이동
import os
os.chdir('/content/sprint-ai03-team04/worktree/wooseok')

# 4. 브랜치 확인
print("\n🔍 현재 브랜치 확인:")
!git branch -a
!git status

# 5. 현재 디렉토리 및 파일 확인
print(f"\n📁 현재 작업 디렉토리: {os.getcwd()}")
print("\n📋 프로젝트 파일 목록:")
!ls -la

# 6. GPU 사용 가능 여부 확인
import torch
print(f"\n🎮 CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 모델: {torch.cuda.get_device_name(0)}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    print(f"CUDA 버전: {torch.version.cuda}")
else:
    print("⚠️  GPU 사용 불가 - CPU 모드로 실행됩니다")

In [ ]:
# 대안 방법: 다른 브랜치로 전환하거나 최신 변경사항 업데이트

# 옵션 1: 이미 클론된 상태에서 다른 브랜치로 전환
def switch_branch(branch_name):
    """다른 브랜치로 전환"""
    print(f"🔄 브랜치 전환: {branch_name}")
    !git checkout {branch_name}
    !git pull origin {branch_name}
    print(f"✅ {branch_name} 브랜치로 전환 완료")

# 옵션 2: 모든 브랜치 확인
def list_all_branches():
    """모든 브랜치 목록 확인"""
    print("📋 사용 가능한 브랜치 목록:")
    !git branch -r

# 옵션 3: 현재 브랜치 최신 상태로 업데이트
def update_current_branch():
    """현재 브랜치 최신 상태로 업데이트"""
    current_branch = !git branch --show-current
    branch_name = current_branch[0].strip() if current_branch else "main"
    print(f"🔄 현재 브랜치 업데이트: {branch_name}")
    !git pull origin {branch_name}
    print("✅ 업데이트 완료")

# 사용 예시 (필요시 주석 해제)
# list_all_branches()
# switch_branch("main")  # main 브랜치로 전환
# update_current_branch()  # 현재 브랜치 업데이트

print("🛠️  브랜치 관리 함수 로드 완료")
print("사용 가능한 함수:")
print("- list_all_branches(): 모든 브랜치 목록 확인")
print("- switch_branch('브랜치명'): 다른 브랜치로 전환")
print("- update_current_branch(): 현재 브랜치 업데이트")

## 3. 데이터셋 설정 및 준비

**⚠️ 중요**: 현재 브랜치에서는 `dataset/` 폴더가 `.gitignore`에 포함되어 있습니다.

1. **데이터셋 업로드**:
   - Google Drive 마운트 또는 직접 업로드
   - 데이터셋 경로 설정 (Git에서 제외됨)
   - 파일 구조 확인

2. **설정 파일 수정**:
   - config.py에서 경로 설정
   - 배치 크기 및 학습 파라미터 조정
   - 브랜치별 설정 차이 확인

3. **브랜치별 차이점**:
   - 개발 브랜치의 최신 기능 사용
   - 실험적 코드 포함 가능성
   - main 브랜치 대비 추가 기능

In [ ]:
# 1. .gitignore 확인 (dataset 폴더 제외 여부)
print("📋 .gitignore 파일 확인:")
!cat /content/sprint-ai03-team04/.gitignore | grep -E "(dataset|data)" || echo "dataset 관련 항목 없음"

# 2. Google Drive 마운트 (데이터셋이 Drive에 있는 경우)
from google.colab import drive
drive.mount('/content/drive')

# 3. 데이터셋 경로 설정
import os

# ⚠️ 중요: dataset 폴더가 .gitignore에 포함되어 있으므로 별도로 데이터셋을 준비해야 합니다
print("\n🚨 데이터셋 준비 방법:")
print("1. Google Drive에서 데이터셋을 복사하거나")
print("2. 직접 업로드하여 사용하세요")

# 옵션 1: Google Drive에서 데이터셋 복사
DRIVE_DATASET_PATH = "/content/drive/MyDrive/your_dataset"  # 실제 경로로 변경
PROJECT_DATASET_PATH = "/content/sprint-ai03-team04/worktree/wooseok/dataset"

# 옵션 2: 직접 업로드한 경우의 데이터셋 경로
DATASET_PATH = "/content/dataset"  # 실제 경로로 변경

print(f"\n📁 데이터셋 경로 옵션:")
print(f"1. Google Drive: {DRIVE_DATASET_PATH}")
print(f"2. 프로젝트 내: {PROJECT_DATASET_PATH}")
print(f"3. 직접 업로드: {DATASET_PATH}")

# 4. 사용할 데이터셋 경로 선택 (아래 중 하나를 선택)
SELECTED_DATASET = DRIVE_DATASET_PATH  # 실제 상황에 맞게 변경

# 5. 데이터셋 구조 확인
if os.path.exists(SELECTED_DATASET):
    print(f"\n✅ 데이터셋 경로 발견: {SELECTED_DATASET}")
    print("📋 데이터셋 구조:")
    !ls -la {SELECTED_DATASET}
    
    # train.json 파일 확인
    if os.path.exists(f"{SELECTED_DATASET}/train.json"):
        print(f"✅ train.json 파일 발견")
    else:
        print("❌ train.json 파일을 찾을 수 없습니다.")
        
    # 이미지 디렉토리 확인
    if os.path.exists(f"{SELECTED_DATASET}/images/train"):
        img_count = len(os.listdir(f"{SELECTED_DATASET}/images/train"))
        print(f"✅ 이미지 디렉토리 발견 ({img_count}개 파일)")
    else:
        print("❌ 이미지 디렉토리를 찾을 수 없습니다.")
else:
    print(f"\n❌ 데이터셋 경로가 존재하지 않습니다: {SELECTED_DATASET}")
    print("📋 데이터셋 준비 가이드:")
    print("1. Google Drive에 데이터셋 업로드")
    print("2. 경로를 SELECTED_DATASET 변수에 설정")
    print("3. 또는 Colab에 직접 업로드")

## 4. 모델 학습 실행

프로젝트 설정이 완료되면 학습을 시작할 수 있습니다:

1. **데이터셋 생성**:
   - 데이터셋 객체 생성
   - 데이터 로더 설정
   - 전처리 파이프라인 확인

2. **학습 실행**:
   - 모델 학습 시작
   - 학습 진행 상황 모니터링
   - 결과 저장

3. **주의사항**:
   - Colab 세션 시간 제한 (12시간)
   - 중간 결과 저장 (체크포인트)
   - 메모리 사용량 모니터링

In [ ]:
# 1. 설정 파일 확인 및 수정
import sys
sys.path.append('/content/sprint-ai03-team04/worktree/wooseok')

# config.py 내용 확인
try:
    from config import cfg
    print("✅ 설정 파일 로드 성공")
    print(f"디바이스: {cfg.DEVICE}")
    print(f"배치 크기: {cfg.BATCH_SIZE}")
    print(f"에포크: {cfg.EPOCHS}")
except Exception as e:
    print(f"❌ 설정 파일 로드 실패: {e}")

# 2. 메모리 사용량 확인
import psutil
import torch

print(f"\n=== 시스템 리소스 확인 ===")
print(f"RAM 사용량: {psutil.virtual_memory().percent}%")
print(f"사용 가능한 RAM: {psutil.virtual_memory().available / (1024**3):.2f} GB")

if torch.cuda.is_available():
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
    print(f"GPU 사용량: {torch.cuda.memory_allocated(0) / (1024**3):.2f} GB")

print("\n=== 환경 설정 완료 ===")
print("이제 데이터셋을 생성하고 학습을 시작할 수 있습니다.")

## 개발 브랜치에서 데이터셋 생성 및 학습 실행

**🌿 현재 브랜치**: `feature/wooseok-baseline-v0`

아래 코드를 사용하여 개발 중인 브랜치에서 데이터셋을 생성하고 학습을 실행할 수 있습니다. 
main 브랜치에 merge하기 전에 여기서 먼저 테스트해보세요.

In [ ]:
# Colab에서 클론한 프로젝트로 데이터셋 생성 및 학습 실행

# 1. 작업 디렉토리 확인 및 설정
import os
import sys

# 프로젝트 디렉토리로 이동
os.chdir('/content/sprint-ai03-team04/worktree/wooseok')
sys.path.append('/content/sprint-ai03-team04/worktree/wooseok')

print(f"현재 작업 디렉토리: {os.getcwd()}")

# 2. 데이터셋 경로 설정 (실제 경로로 변경하세요)
DATASET_JSON = "/content/dataset/train.json"  # 실제 경로로 변경
DATASET_IMAGES = "/content/dataset/images/train"  # 실제 경로로 변경

# 또는 Google Drive에서 가져오는 경우:
# DATASET_JSON = "/content/drive/MyDrive/your_dataset/train.json"
# DATASET_IMAGES = "/content/drive/MyDrive/your_dataset/images/train"

# 3. 데이터셋 생성
try:
    from dataset import create_colab_dataset
    
    dataset = create_colab_dataset(
        json_path=DATASET_JSON,
        img_dir=DATASET_IMAGES
    )
    
    if dataset is not None:
        print(f"✅ 데이터셋 생성 성공: {len(dataset)} 개의 샘플")
        
        # 4. 학습 실행
        from train import train_with_dataset
        print("🚀 학습 시작...")
        train_with_dataset(dataset)
        
    else:
        print("❌ 데이터셋 생성 실패")
        print("데이터셋 경로를 확인하세요:")
        print(f"JSON 파일: {DATASET_JSON}")
        print(f"이미지 디렉토리: {DATASET_IMAGES}")
        
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("프로젝트 파일이 올바르게 클론되었는지 확인하세요.")

In [ ]:
# 대안 방법: 단계별로 학습 실행하기

# 1. 프로젝트 디렉토리 확인
import os
import sys
os.chdir('/content/sprint-ai03-team04/worktree/wooseok')
sys.path.append('/content/sprint-ai03-team04/worktree/wooseok')

print("📁 프로젝트 파일 확인:")
required_files = ['train.py', 'dataset.py', 'config.py', 'trainer.py']
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - 파일이 없습니다")

# 2. 데이터셋 파일 직접 지정
# Google Drive 또는 업로드된 데이터셋 경로를 정확히 입력하세요
json_path = "/content/drive/MyDrive/your_dataset/train.json"  # 실제 경로로 변경
img_dir = "/content/drive/MyDrive/your_dataset/images/train"  # 실제 경로로 변경

print(f"\n📊 데이터셋 경로:")
print(f"JSON: {json_path}")
print(f"Images: {img_dir}")

# 3. 경로 존재 확인
json_exists = os.path.exists(json_path)
img_exists = os.path.exists(img_dir)

print(f"\n📋 경로 확인:")
print(f"JSON 파일 존재: {json_exists}")
print(f"이미지 디렉토리 존재: {img_exists}")

if json_exists and img_exists:
    print(f"이미지 개수: {len(os.listdir(img_dir))}")
    
    # 4. 데이터셋 생성 및 학습
    try:
        from dataset import PillDetectionDataset
        from train import train_with_dataset
        
        dataset = PillDetectionDataset(
            json_file=json_path,
            img_dir=img_dir
        )
        
        print(f"✅ 데이터셋 로드 완료: {len(dataset)} 샘플")
        
        # 학습 실행
        print("🚀 학습 시작...")
        train_with_dataset(dataset)
        
    except Exception as e:
        print(f"❌ 학습 실행 중 오류: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("❌ 데이터셋 경로를 확인하고 다시 시도하세요.")
    print("Google Drive를 마운트했는지, 경로가 올바른지 확인하세요.")

## 5. 브랜치 개발 및 문제 해결

### 🌿 브랜치 관련 팁

1. **현재 브랜치 확인**:
   - `!git branch --show-current`
   - `!git status`

2. **브랜치 전환**:
   - `!git checkout main` (main 브랜치로)
   - `!git checkout feature/wooseok-baseline-v0` (개발 브랜치로)

3. **최신 변경사항 가져오기**:
   - `!git pull origin feature/wooseok-baseline-v0`

### 📁 데이터셋 관리

1. **Git에서 제외된 파일들**:
   - `dataset/` 폴더 (`.gitignore`에 포함)
   - `__pycache__/` 폴더
   - 로그 파일, 모델 체크포인트

2. **데이터셋 준비 방법**:
   - Google Drive 업로드 후 마운트
   - 직접 Colab에 업로드
   - 외부 링크에서 다운로드

### 🚨 개발 브랜치 주의사항

- 개발 중인 코드로 예상치 못한 오류 가능
- 실험적 기능 포함 가능성
- main 브랜치 대비 불안정할 수 있음
- 테스트 후 이상 없으면 main으로 merge

### 🔄 merge 전 체크리스트

- [ ] 코드 정상 실행 확인
- [ ] 기본 기능 테스트 완료
- [ ] 오류 메시지 없음
- [ ] 성능 저하 없음
- [ ] 문서 업데이트 필요시 반영

## 🚨 CUDA 메모리 오류 해결 방법

**"CUDA error: an illegal memory access was encountered"** 오류가 발생하는 경우:

### 📋 일반적인 원인들:
1. **GPU 메모리 부족** - 배치 크기가 너무 큼
2. **텐서 크기 불일치** - 모델 입력과 실제 데이터 크기 차이
3. **CUDA 버전 호환성** - PyTorch와 CUDA 버전 불일치
4. **메모리 누수** - 이전 실행에서 남은 GPU 메모리

### 🔧 해결 방법들:
1. **GPU 메모리 정리 후 재시작**
2. **배치 크기 줄이기**
3. **디버깅 모드 활성화**
4. **데이터 로더 worker 수 조정**

## 🔄 PyTorch 순환 Import 오류 해결

**"partially initialized module 'torch._dynamo' has no attribute 'config'"** 오류 해결:

### 📋 원인 분석:
1. **순환 Import** - 프로젝트 파일들 간의 circular import
2. **PyTorch 버전 문제** - 버전 호환성 이슈
3. **모듈 초기화 오류** - 모듈이 완전히 로드되기 전 접근
4. **캐시된 모듈** - 이전 실행의 잘못된 캐시

### 🔧 해결 방법:
1. **파이썬 모듈 캐시 정리**
2. **Import 순서 조정**
3. **PyTorch 재설치**
4. **런타임 재시작**

In [ ]:
# PyTorch 순환 Import 오류 해결을 위한 종합 솔루션

import sys
import os
import importlib
import gc

def clean_python_modules():
    """Python 모듈 캐시 정리"""
    print("🧹 Python 모듈 캐시 정리 중...")
    
    # 1. 프로젝트 관련 모듈 제거
    modules_to_remove = []
    for module_name in list(sys.modules.keys()):
        if any(keyword in module_name for keyword in ['config', 'dataset', 'train', 'trainer', 'torch._dynamo']):
            modules_to_remove.append(module_name)
    
    for module_name in modules_to_remove:
        if module_name in sys.modules:
            del sys.modules[module_name]
            print(f"  ✅ {module_name} 모듈 제거")
    
    # 2. 가비지 컬렉션
    gc.collect()
    print("✅ 모듈 캐시 정리 완료")

def fix_pytorch_import():
    """PyTorch Import 문제 해결"""
    print("🔧 PyTorch Import 문제 해결 중...")
    
    # 1. 환경 변수 설정
    os.environ['PYTHONDONTWRITEBYTECODE'] = '1'  # .pyc 파일 생성 방지
    
    # 2. PyTorch 모듈 강제 재로드
    try:
        import torch
        importlib.reload(torch)
        print("✅ PyTorch 모듈 재로드 완료")
    except Exception as e:
        print(f"❌ PyTorch 재로드 실패: {e}")
    
    # 3. torch._dynamo 직접 확인
    try:
        import torch._dynamo
        if hasattr(torch._dynamo, 'config'):
            print("✅ torch._dynamo.config 정상 접근 가능")
        else:
            print("❌ torch._dynamo.config 속성 없음")
    except Exception as e:
        print(f"❌ torch._dynamo 접근 실패: {e}")

def check_circular_imports():
    """순환 import 확인"""
    print("🔍 순환 import 확인 중...")
    
    # 프로젝트 디렉토리 설정
    project_dir = '/content/sprint-ai03-team04/worktree/wooseok'
    
    if os.path.exists(project_dir):
        os.chdir(project_dir)
        
        # 프로젝트 파일들 확인
        project_files = ['config.py', 'dataset.py', 'train.py', 'trainer.py']
        
        for file in project_files:
            if os.path.exists(file):
                print(f"📄 {file} 파일 존재")
                
                # 각 파일의 import 구문 확인
                try:
                    with open(file, 'r', encoding='utf-8') as f:
                        content = f.read()
                        imports = [line.strip() for line in content.split('\n') 
                                 if line.strip().startswith('import ') or line.strip().startswith('from ')]
                        
                        print(f"  Import 구문 ({len(imports)}개):")
                        for imp in imports[:5]:  # 처음 5개만 표시
                            print(f"    {imp}")
                        if len(imports) > 5:
                            print(f"    ... 및 {len(imports) - 5}개 더")
                            
                except Exception as e:
                    print(f"  ❌ 파일 읽기 실패: {e}")
            else:
                print(f"❌ {file} 파일 없음")
    else:
        print(f"❌ 프로젝트 디렉토리 없음: {project_dir}")

def reinstall_pytorch():
    """PyTorch 재설치"""
    print("🔄 PyTorch 재설치 중...")
    
    # 현재 PyTorch 버전 확인
    try:
        import torch
        print(f"현재 PyTorch 버전: {torch.__version__}")
        print(f"CUDA 버전: {torch.version.cuda}")
    except:
        print("PyTorch 버전 확인 불가")
    
    # 재설치 명령어 제공
    print("\n🛠️  PyTorch 재설치 명령어:")
    print("다음 셀에서 실행하세요:")
    print("!pip uninstall torch torchvision torchaudio -y")
    print("!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")

# 실행
print("🚨 PyTorch 순환 Import 오류 해결 시작")
clean_python_modules()
fix_pytorch_import()
check_circular_imports()
reinstall_pytorch()

print("\n🎯 권장 해결 순서:")
print("1. 런타임 재시작 (가장 효과적)")
print("2. PyTorch 재설치")
print("3. 프로젝트 파일 import 구조 확인")
print("4. 안전한 import 순서로 재실행")

In [ ]:
# PyTorch 재설치 (순환 import 오류 해결)

print("🔄 PyTorch 완전 재설치 시작...")

# 1. 기존 PyTorch 완전 제거
print("1️⃣ 기존 PyTorch 제거 중...")
!pip uninstall torch torchvision torchaudio -y

# 2. 시스템 캐시 정리
print("\n2️⃣ 시스템 캐시 정리 중...")
!pip cache purge

# 3. PyTorch 재설치 (CUDA 11.8 버전)
print("\n3️⃣ PyTorch 재설치 중...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 4. 설치 확인
print("\n4️⃣ 설치 확인 중...")
try:
    import torch
    import torchvision
    print(f"✅ PyTorch 버전: {torch.__version__}")
    print(f"✅ TorchVision 버전: {torchvision.__version__}")
    print(f"✅ CUDA 버전: {torch.version.cuda}")
    print(f"✅ CUDA 사용 가능: {torch.cuda.is_available()}")
    
    # torch._dynamo 확인
    try:
        import torch._dynamo
        if hasattr(torch._dynamo, 'config'):
            print("✅ torch._dynamo.config 정상 접근 가능")
        else:
            print("⚠️  torch._dynamo.config 속성 없음")
    except Exception as e:
        print(f"⚠️  torch._dynamo 접근 문제: {e}")
        
except Exception as e:
    print(f"❌ PyTorch 설치 확인 실패: {e}")

print("\n🎯 다음 단계:")
print("1. 런타임 재시작 (권장)")
print("2. 프로젝트 다시 클론")
print("3. 학습 코드 재실행")
print("4. 여전히 문제 시 순환 import 확인 필요")

In [ ]:
# 안전한 Import 순서로 학습 실행 (순환 import 방지)

import sys
import os
import gc

print("🛡️  안전한 Import 순서로 학습 실행")

# 1. 환경 초기화
print("1️⃣ 환경 초기화 중...")
os.environ['PYTHONDONTWRITEBYTECODE'] = '1'
gc.collect()

# 2. 프로젝트 디렉토리 설정
project_dir = '/content/sprint-ai03-team04/worktree/wooseok'
os.chdir(project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)

print(f"📁 작업 디렉토리: {os.getcwd()}")

# 3. 안전한 순서로 모듈 Import
print("\n2️⃣ 안전한 순서로 모듈 Import 중...")

try:
    # 기본 라이브러리 먼저
    print("  🔹 기본 라이브러리 import...")
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader
    
    # 프로젝트 모듈들 순서대로
    print("  🔹 config 모듈 import...")
    from config import cfg
    
    print("  🔹 dataset 모듈 import...")
    from dataset import PillDetectionDataset, create_colab_dataset
    
    print("  🔹 model 모듈 import...")
    from models.yolo_model import create_model
    
    print("  🔹 trainer 모듈 import...")
    from trainer import Trainer
    
    print("  🔹 train 모듈 import...")
    from train import train_with_dataset
    
    print("✅ 모든 모듈 import 성공")
    
except ImportError as e:
    print(f"❌ Import 오류: {e}")
    print("🔄 권장 조치:")
    print("1. 런타임 재시작")
    print("2. PyTorch 재설치")
    print("3. 프로젝트 파일 확인")
    
except Exception as e:
    print(f"❌ 예상치 못한 오류: {e}")
    import traceback
    traceback.print_exc()

# 4. 안전한 설정
print("\n3️⃣ 안전한 설정 적용 중...")
try:
    # 안전한 설정으로 변경
    cfg.BATCH_SIZE = 2
    cfg.NUM_WORKERS = 0
    cfg.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    print(f"✅ 배치 크기: {cfg.BATCH_SIZE}")
    print(f"✅ Worker 수: {cfg.NUM_WORKERS}")
    print(f"✅ 디바이스: {cfg.DEVICE}")
    
except Exception as e:
    print(f"❌ 설정 적용 실패: {e}")

# 5. 데이터셋 경로 설정
print("\n4️⃣ 데이터셋 경로 설정...")
DATASET_JSON = "/content/drive/MyDrive/your_dataset/train.json"  # 실제 경로로 변경
DATASET_IMAGES = "/content/drive/MyDrive/your_dataset/images/train"  # 실제 경로로 변경

print(f"📊 JSON 파일: {DATASET_JSON}")
print(f"🖼️  이미지 디렉토리: {DATASET_IMAGES}")

# 6. 학습 실행 (안전 모드)
print("\n5️⃣ 학습 실행 준비 완료")
print("🎯 다음 단계:")
print("1. 위의 경로를 실제 데이터셋 경로로 변경")
print("2. 아래 코드 주석 해제하여 학습 실행")

# 아래 주석을 해제하면 학습 실행
"""
try:
    # 데이터셋 생성
    dataset = create_colab_dataset(
        json_path=DATASET_JSON,
        img_dir=DATASET_IMAGES
    )
    
    if dataset is not None:
        print(f"✅ 데이터셋 생성 성공: {len(dataset)} 샘플")
        
        # 학습 실행
        print("🚀 안전 모드로 학습 시작...")
        train_with_dataset(dataset)
        
    else:
        print("❌ 데이터셋 생성 실패")
        
except Exception as e:
    print(f"❌ 학습 실행 중 오류: {e}")
    import traceback
    traceback.print_exc()
"""

In [ ]:
# CUDA 메모리 오류 해결을 위한 디버깅 및 수정 코드

import os
import torch
import gc

def fix_cuda_memory_error():
    """CUDA 메모리 오류 해결을 위한 종합 솔루션"""
    print("🔧 CUDA 메모리 오류 해결 중...")
    
    # 1. GPU 메모리 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("✅ GPU 메모리 캐시 정리 완료")
    
    # 2. 디버깅 모드 활성화
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    print("✅ CUDA 디버깅 모드 활성화")
    
    # 3. GPU 메모리 상태 확인
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        total_memory = torch.cuda.get_device_properties(device).total_memory
        allocated_memory = torch.cuda.memory_allocated(device)
        cached_memory = torch.cuda.memory_reserved(device)
        
        print(f"\n📊 GPU 메모리 상태:")
        print(f"총 메모리: {total_memory / (1024**3):.2f} GB")
        print(f"할당된 메모리: {allocated_memory / (1024**3):.2f} GB")
        print(f"캐시된 메모리: {cached_memory / (1024**3):.2f} GB")
        print(f"사용 가능한 메모리: {(total_memory - allocated_memory) / (1024**3):.2f} GB")
    
    # 4. 권장 설정 출력
    print(f"\n⚙️  권장 설정:")
    print(f"배치 크기: 4 이하 (현재 A100-40GB 기준)")
    print(f"Worker 수: 0-2 (Colab 환경)")
    print(f"Mixed precision: 사용 권장")

def create_safe_config():
    """안전한 학습 설정 생성"""
    print("🛡️  안전한 학습 설정 생성...")
    
    # config.py 수정을 위한 안전한 값들
    safe_config = {
        'BATCH_SIZE': 2,          # 배치 크기 대폭 감소
        'NUM_WORKERS': 0,         # Worker 수 0으로 설정
        'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu',
        'MIXED_PRECISION': True,  # Mixed precision 활성화
        'GRADIENT_CLIP': 1.0,     # 그래디언트 클리핑
    }
    
    print("📋 안전한 설정값:")
    for key, value in safe_config.items():
        print(f"  {key}: {value}")
    
    return safe_config

def test_simple_tensor_operations():
    """간단한 텐서 연산으로 CUDA 상태 테스트"""
    print("🧪 CUDA 상태 테스트 중...")
    
    try:
        if torch.cuda.is_available():
            # 간단한 텐서 연산 테스트
            device = torch.device('cuda')
            x = torch.randn(10, 10).to(device)
            y = torch.randn(10, 10).to(device)
            z = torch.mm(x, y)
            print("✅ 기본 CUDA 텐서 연산 정상")
            
            # 메모리 해제
            del x, y, z
            torch.cuda.empty_cache()
            
        else:
            print("❌ CUDA 사용 불가")
            
    except Exception as e:
        print(f"❌ CUDA 테스트 실패: {e}")
        return False
    
    return True

# 실행
print("🚨 CUDA 메모리 오류 해결 시작")
fix_cuda_memory_error()
safe_config = create_safe_config()
cuda_test_passed = test_simple_tensor_operations()

if cuda_test_passed:
    print("\n✅ CUDA 상태 정상 - 안전한 설정으로 학습 재시도 가능")
else:
    print("\n❌ CUDA 상태 불안정 - 런타임 재시작 권장")
    
print("\n🔄 다음 단계:")
print("1. 런타임 재시작 (필요시)")
print("2. 배치 크기를 2로 설정")
print("3. NUM_WORKERS를 0으로 설정")
print("4. 학습 재시도")

In [ ]:
# CUDA 오류 해결 후 안전한 학습 실행

import os
import sys
import torch
import gc

# 1. 환경 설정
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # 디버깅 모드
os.chdir('/content/sprint-ai03-team04/worktree/wooseok')
sys.path.append('/content/sprint-ai03-team04/worktree/wooseok')

# 2. GPU 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 GPU 메모리 정리 완료")

# 3. 안전한 설정으로 config 임시 수정
try:
    from config import cfg
    
    # 원본 설정 백업
    original_batch_size = cfg.BATCH_SIZE
    original_num_workers = cfg.NUM_WORKERS
    
    # 안전한 설정으로 임시 변경
    cfg.BATCH_SIZE = 2          # 배치 크기 대폭 감소
    cfg.NUM_WORKERS = 0         # Worker 수 0으로 설정
    cfg.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    print("⚙️  안전한 설정으로 변경:")
    print(f"배치 크기: {original_batch_size} → {cfg.BATCH_SIZE}")
    print(f"Worker 수: {original_num_workers} → {cfg.NUM_WORKERS}")
    print(f"디바이스: {cfg.DEVICE}")
    
except Exception as e:
    print(f"❌ 설정 로드 실패: {e}")

# 4. 데이터셋 경로 설정 (실제 경로로 변경)
DATASET_JSON = "/content/drive/MyDrive/your_dataset/train.json"  # 실제 경로로 변경
DATASET_IMAGES = "/content/drive/MyDrive/your_dataset/images/train"  # 실제 경로로 변경

# 5. 안전한 학습 실행
try:
    print("🚀 안전한 학습 시작...")
    
    # 데이터셋 생성
    from dataset import create_colab_dataset
    dataset = create_colab_dataset(
        json_path=DATASET_JSON,
        img_dir=DATASET_IMAGES
    )
    
    if dataset is not None:
        print(f"✅ 데이터셋 생성 성공: {len(dataset)} 샘플")
        
        # 안전한 학습 실행
        from train import train_with_dataset
        
        # 추가 안전 장치
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
        print("🛡️  안전 모드로 학습 시작...")
        train_with_dataset(dataset)
        
    else:
        print("❌ 데이터셋 생성 실패")
        print("데이터셋 경로를 확인하세요:")
        print(f"JSON: {DATASET_JSON}")
        print(f"Images: {DATASET_IMAGES}")
        
except RuntimeError as e:
    if "CUDA" in str(e):
        print(f"❌ CUDA 오류 재발생: {e}")
        print("\n🔄 권장 조치:")
        print("1. 런타임 > 런타임 재시작")
        print("2. 배치 크기를 1로 더 줄이기")
        print("3. CPU 모드로 전환 (cfg.DEVICE = 'cpu')")
    else:
        print(f"❌ 기타 오류: {e}")
        
except Exception as e:
    print(f"❌ 예상치 못한 오류: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # 메모리 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("🧹 학습 후 메모리 정리 완료")

In [ ]:
# 🚨 긴급 대안: CPU 전용 학습 (CUDA 오류 지속 시)

import os
import sys
import torch

print("🔄 CPU 전용 모드로 전환...")

# 1. CUDA 완전 비활성화
os.environ['CUDA_VISIBLE_DEVICES'] = ''
torch.cuda.is_available = lambda: False

# 2. 프로젝트 설정
os.chdir('/content/sprint-ai03-team04/worktree/wooseok')
sys.path.append('/content/sprint-ai03-team04/worktree/wooseok')

# 3. CPU 전용 설정
try:
    from config import cfg
    
    cfg.DEVICE = 'cpu'
    cfg.BATCH_SIZE = 4          # CPU에서는 배치 크기 조금 늘려도 됨
    cfg.NUM_WORKERS = 2         # CPU에서는 worker 사용 가능
    cfg.EPOCHS = 2              # 테스트를 위해 에포크 수 줄이기
    
    print("💻 CPU 전용 설정:")
    print(f"디바이스: {cfg.DEVICE}")
    print(f"배치 크기: {cfg.BATCH_SIZE}")
    print(f"Worker 수: {cfg.NUM_WORKERS}")
    print(f"에포크: {cfg.EPOCHS}")
    
except Exception as e:
    print(f"❌ 설정 로드 실패: {e}")

# 4. CPU 전용 학습 실행
print("\n⚠️  주의: CPU 모드는 매우 느립니다 (테스트 목적)")
print("실제 학습은 CUDA 오류 해결 후 GPU에서 실행하세요")

# 사용자 확인
print("\n🤔 CPU 모드로 계속 진행하시겠습니까?")
print("계속하려면 아래 주석을 해제하고 실행하세요:")

# 아래 주석을 해제하면 CPU 모드로 실행
"""
try:
    # 데이터셋 경로 설정
    DATASET_JSON = "/content/drive/MyDrive/your_dataset/train.json"
    DATASET_IMAGES = "/content/drive/MyDrive/your_dataset/images/train"
    
    from dataset import create_colab_dataset
    dataset = create_colab_dataset(
        json_path=DATASET_JSON,
        img_dir=DATASET_IMAGES
    )
    
    if dataset is not None:
        print(f"✅ 데이터셋 생성 성공: {len(dataset)} 샘플")
        
        from train import train_with_dataset
        print("💻 CPU 모드로 학습 시작... (매우 느림)")
        train_with_dataset(dataset)
        
    else:
        print("❌ 데이터셋 생성 실패")
        
except Exception as e:
    print(f"❌ CPU 학습 중 오류: {e}")
    import traceback
    traceback.print_exc()
"""

print("\n🎯 권장 해결책:")
print("1. 런타임 > 런타임 재시작")
print("2. GPU 메모리 정리 후 재시도")
print("3. 배치 크기를 1로 설정")
print("4. 이전 셀의 '안전한 학습 실행' 코드 사용")

In [ ]:
# 유용한 유틸리티 함수들

import os
import shutil
import psutil
import torch
from datetime import datetime

def check_system_resources():
    """시스템 리소스 사용량 확인"""
    print("=== 시스템 리소스 현황 ===")
    
    # CPU 및 메모리
    cpu_percent = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory()
    
    print(f"🖥️  CPU 사용량: {cpu_percent}%")
    print(f"💾 메모리 사용량: {memory.percent}%")
    print(f"📊 사용 가능한 메모리: {memory.available / (1024**3):.2f} GB")
    
    # GPU 정보
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory
        gpu_allocated = torch.cuda.memory_allocated(0)
        gpu_cached = torch.cuda.memory_reserved(0)
        
        print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
        print(f"📈 GPU 메모리 총량: {gpu_memory / (1024**3):.2f} GB")
        print(f"🔢 GPU 할당된 메모리: {gpu_allocated / (1024**3):.2f} GB")
        print(f"💽 GPU 캐시된 메모리: {gpu_cached / (1024**3):.2f} GB")
    else:
        print("❌ GPU를 사용할 수 없습니다")

def save_model_to_drive(model_path, drive_path):
    """모델을 Google Drive에 저장"""
    try:
        if not os.path.exists(drive_path):
            os.makedirs(drive_path)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = os.path.join(drive_path, f"model_{timestamp}.pth")
        
        shutil.copy2(model_path, save_path)
        print(f"✅ 모델 저장 완료: {save_path}")
        return save_path
    except Exception as e:
        print(f"❌ 모델 저장 실패: {e}")
        return None

def clean_gpu_cache():
    """GPU 캐시 정리"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("🧹 GPU 캐시 정리 완료")
    else:
        print("❌ GPU를 사용할 수 없습니다")

def quick_dataset_check(json_path, img_dir):
    """데이터셋 빠른 확인"""
    print("=== 데이터셋 빠른 확인 ===")
    
    # JSON 파일 확인
    if os.path.exists(json_path):
        print(f"✅ JSON 파일 존재: {json_path}")
        
        import json
        with open(json_path, 'r') as f:
            data = json.load(f)
            print(f"📊 이미지 개수: {len(data['images'])}")
            print(f"🏷️  어노테이션 개수: {len(data['annotations'])}")
            print(f"🎯 카테고리 개수: {len(data['categories'])}")
    else:
        print(f"❌ JSON 파일 없음: {json_path}")
    
    # 이미지 디렉토리 확인
    if os.path.exists(img_dir):
        img_count = len([f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        print(f"✅ 이미지 디렉토리 존재: {img_dir}")
        print(f"🖼️  이미지 파일 개수: {img_count}")
    else:
        print(f"❌ 이미지 디렉토리 없음: {img_dir}")

# 사용 예시
print("🔧 유틸리티 함수 로드 완료")
print("사용 가능한 함수들:")
print("- check_system_resources(): 시스템 리소스 확인")
print("- save_model_to_drive(model_path, drive_path): 모델 Drive 저장")
print("- clean_gpu_cache(): GPU 캐시 정리")
print("- quick_dataset_check(json_path, img_dir): 데이터셋 빠른 확인")

# 현재 리소스 상태 확인
check_system_resources()